In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import math

In [2]:
class TokenEmbedding(nn.Embedding):#这个类用于将离散的词汇索引（通常是整数）映射到连续的密集向量
    def __init__(self,vocab_size,d_model):#d_model是qkv空间的维度
        super(TokenEmbedding,self).__init__(vocab_size,d_model,padding_idx=1)#padding_idx=1不进行梯度更新

怎么区分广播机制
写多了广播机制我就分不清维数了

In [3]:
class PositionalEmbedding(nn.Module):
    def __init__(self,d_model,maxlen,device):
        super(PositionalEmbedding,self).__init__()
        self.encoding=torch.zeros(maxlen,d_model,device=device)
        self.encoding.requires_grad_(False)

        pos=torch.arange(0,maxlen,device=device)

        pos=pos.float().unsqueeze(1)#加一维,变成向量

        _2i=torch.arange(0,d_model,2,device=device)

        self.encoding[:,0::2] = torch.sin(pos/(10000**(_2i/d_model)))#broadcast机制
        #   _2i   (d_model//2, 1 )
        #   pos   (maxlen,  1 )
        self.encoding[:,1::2] = torch.cos(pos/(10000**(_2i/d_model)))

    def forward(self,x):
        seq_leng=x.shape[1]#x 的 形状 ： batch sequenceline dimension
        return self.encoding[:seq_leng,:]#选择前seq_leng行，所有列

In [4]:
class TransformerEmbedding(nn.Module):
    def __init__(self,vocab_size,d_model,maxlen,dropout,device):
        super(TransformerEmbedding,self).__init__()
        self.tok_emb=TokenEmbedding(vocab_size,d_model)
        self.pos_emb=PositionalEmbedding(d_model,maxlen,device)
        self.drop=nn.Dropout(dropout)
    
    def forward(self,x):
        tok_emb=self.tok_emb(x)
        pos_emb=self.pos_emb(x)
        return self.drop(tok_emb+pos_emb)#将token嵌入和pos嵌入相加，再通过dropout
        

![image.png](./picture/layer.png)

In [5]:
class LayerNorm(nn.Module):#图像用BatchNorm比较多 
    def __init__(self, d_model, eps=1e-10):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased=False, keepdim=True)
        out = (x - mean) / torch.sqrt(var + self.eps)
        out = self.gamma * out + self.beta
        return out

![image.png](./picture/ffn.png)

In [6]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model,hidden,dropout=0.1):
        super(PositionwiseFeedForward,self).__init__()
        self.fc1=nn.Linear(d_model,hidden)
        self.fc2=nn.Linear(hidden,d_model)
        self.dropout=nn.Dropout(dropout)
    
    def forward(self,x):
        feature_map=self.fc1(x)
        feature_map=F.relu(feature_map)
        feature_map=self.fc2(feature_map)
        feature_map=self.dropout(feature_map)#在哪里dropout
        return feature_map


In [7]:
class GroupQueryAttention(nn.Module):
    def __init__(self,d_model,n_heads,n_groups):
        super(GroupQueryAttention,self).__init__()
        self.d_model=d_model
        self.n_heads=n_heads
        self.n_groups=n_groups

        assert d_model%n_heads == 0
        self.n_heads_groups=self.n_heads//self.n_groups#整除操作符
        self.head_dim=d_model//self.n_heads

        self.w_q=nn.Linear(d_model,d_model)
        self.w_l=nn.Linear(d_model,self.n_groups*self.head_dim)#这里的维数
        self.w_v=nn.Linear(d_model,self.n_groups*self.head_dim)
        
        self.w_combine=nn.Linear(d_model,d_model)#将多头输出合并为单头输出
        self.softmax=nn.Softmax(dim=-1)
    """
        1. 使用`[:,:,None,:,:]`在第二个维度（即头数维度）上添加一个新的维度，大小为1。  
        2. 使用`expand`将新维度扩展到`n_groups`和`n_heads_groups`。  
        3. 使用`contiguous`确保张量是连续的（这有助于后续的`view`操作）。  
        4. 使用`view`改变张量的形状，使其变为`[batch, n_groups * n_heads_groups, time, head_dim]`。
    """
    def expand(self,data):
        batch,time=data.shape[0],data.shape[2]
        data=data[:,:,None,:,:].expand(batch,self.n_groups,self.n_heads_groups,time,self.head_dim).contiguous()
        data=data.view(batch,self.n_groups*self.n_heads_groups,time,self.head_dim)
        
    def forward(self,q,k,v,mask=None):
        q=self.w_q(q)
        k=self.w_k(k)
        v=self.w_v(v)

        batch=q.shape[0]    
        q=q.view(batch,-1,self.n_groups*self.n_heads_groups,self.head_dim).permute(0,2,1,3)#permute 方法用于重新排列张量的维度。
        k=k.view(batch,-1,self.n_groups,self.head_dim).permute(0,2,1,3)
        v=v.view(batch,-1,self.n_groups,self.head_dim).permute(0,2,1,3)

        k=self.expand(k)
        v=self.expand(v)
        score=q@k.transpose(2,3)/math.sqrt(self.head_dim)#@是矩阵乘法matmul
        if mask is not None:
            score=score.masked_fill(mask==0,-1e9)
        score=self.softmax(score)@v
        #.contiguous 确保张量是连续的（contiguous）在内存中。在 PyTorch 中，当张量经过某些操作（如 transpose、permute 等）后，其内存可能不再是连续的，这可能导致后续的某些操作（如 view）失败。
        score=score.permute(0,2,1,3).contiguous().view(batch,-1,self.d_model)
        output=self.w_combine(score)
        return output
    


为啥要通道交换？
这里直接乘的话会有什么后果？

n_head是头个数

怎么去确定每个矩阵的维度，我感觉写着写着就乱了----根据论文复现代码

包括下面的矩阵乘法

transpose没搞懂

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_model,n_head):
        super(MultiHeadAttention,self).__init__()
        self.n_head = n_head
        self.d_model = d_model
        self.w_q = nn.Linear(d_model, d_model)#query
        self.w_k = nn.Linear(d_model, d_model)#key
        self.w_v = nn.Linear(d_model, d_model)#value
        self.w_combine = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None):
        batch, time, dimension = q.shape
        n_d = self.d_model // self.n_head#整除

        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)
        
        # q = q.view(batch, time, self.n_head, n_d).permute(0, 2, 1, 3)#通道维度交换
        # k = k.view(batch, time, self.n_head, n_d).permute(0, 2, 1, 3)
        # v = v.view(batch, time, self.n_head, n_d).permute(0, 2, 1, 3)

        # score = q @ k.transpose(2, 3) / math.sqrt(n_d)

        score=q@k.transpose()/math.sqrt(n_d)

        if mask is not None:
            # mask = torch.tril(torch.ones(time, time, dtype=bool))
            score = score.masked_fill(mask == 0, -1e9)#在softmax时把0的地方设置为负无穷
        score = self.softmax(score) @ v

        score = score.permute(0, 2, 1, 3).contiguous().view(batch, time, dimension)#通道维度交换回来    

        output = self.w_combine(score)
        return output
    

In [9]:
class EncoderLayer(nn.Module):
    def __init__(self,d_model,ffn_hidden,n_head,drop_prob)->None:
        super(EncoderLayer,self).__init__()
        self.attention=MultiHeadAttention(d_model,n_head)
        self.norm1=LayerNorm(d_model)
        self.drop1=nn.Dropout(drop_prob)

        self.ffn=PositionwiseFeedForward(d_model,ffn_hidden,drop_prob)
        self.norm2=LayerNorm(d_model)
        self.drop2=nn.Dropout(drop_prob)

    def forward(self,x,mask=None):
        _x=x
        x=self.attention(x,x,x,mask)
        x=self.drop1(x)
        x=self.norm1(x+_x)

        _x=x
        x=self.ffn(x)
        x=self.drop2(x)
        x=self.norm2(x+_x)
        return x
    

In [10]:
class DecoderLayer(nn.Module):
    def __init__(self,d_model,ffn_hidden,n_head,drop_prob)->None:
        super(DecoderLayer,self).__init__()
        self.attention1=MultiHeadAttention(d_model,n_head)
        self.norm1=LayerNorm(d_model)
        self.drop1=nn.Dropout(drop_prob)

        self.attention2=MultiHeadAttention(d_model,n_head)
        self.norm2=LayerNorm(d_model)
        self.drop2=nn.Dropout(drop_prob)

        self.ffn=PositionwiseFeedForward(d_model,ffn_hidden,drop_prob)
        self.norm3=LayerNorm(d_model)
        self.drop3=nn.Dropout(drop_prob)

    def forward(self,dec,enc,t_mask,s_mask):
        _x=dec
        x=self.attention1(dec,dec,dec,t_mask)#下三角掩码
        x=self.drop1(x)
        x=self.norm1(x+_x)

        if enc is not None:
            _x=x
            x=self.attention2(x,enc,enc,s_mask)
            x=self.drop2(x)
            x=self.norm2(x+_x)
        _x=x
        x=self.ffn(x)
        x=self.drop3(x)
        x=self.norm3(x+_x)
        return x


列表推导式

In [11]:
class Encoder(nn.Module):
    def __init__(self,enc_voc_size,max_len,d_model,ffn_hidden,n_head,n_layer,drop_prob,device):
        super(Encoder,self).__init__()
        self.embedding=TransformerEmbedding(enc_voc_size,d_model,max_len,drop_prob,device)
        #这行代码实际上使用了列表推导式（list comprehension）而不是显式的for循环，但它背后的概念是相同的：重复执行某个操作（在这个例子中是创建EncoderLayer对象）n_layer次。
        self.layers=nn.ModuleList([EncoderLayer(d_model,ffn_hidden,n_head,drop_prob)
                                   for _ in range(n_layer)
                                   ])

    def forward(self,x,s_mask):
        x=self.embedding(x)
        for layer in self.layers:
            x=layer(x,s_mask)
        return x
    

In [12]:
class Decoder(nn.Module):
    def __init__(self,dec_voc_size,max_len,d_model,ffn_hidden,n_head,n_layer,drop_prob,device):
        super(Decoder,self).__init__()
        self.embedding=TransformerEmbedding(dec_voc_size,d_model,max_len,drop_prob,device)
        self.layers=nn.ModuleList([DecoderLayer(d_model,ffn_hidden,n_head,drop_prob)
                                   for _ in range(n_layer)
                                   ])
        self.fc=nn.Linear(d_model,dec_voc_size)

    def forward(self,dec,enc,t_mask,s_mask):
        dec=self.embedding(dec)
        for layer in self.layers:
            dec=layer(dec,enc,t_mask,s_mask)
        dec=self.fc(dec)

        return dec
    

In [13]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_pad_idx,
        trg_pad_idx,
        enc_voc_size,
        dec_voc_size,
        max_len,
        d_model,
        n_heads,
        ffn_hidden,
        n_layers,
        drop_prob,
        device,
    ):
        super(Transformer, self).__init__()
        self.encoder = Encoder(
            enc_voc_size,
            max_len,
            d_model,
            ffn_hidden,
            n_heads,
            n_layers,
            drop_prob,
            device,
        )
        self.decoder = Decoder(
            dec_voc_size,
            max_len,
            d_model,
            ffn_hidden,
            n_heads,
            n_layers,
            drop_prob,
            device,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
    def make_pad_mask(self,q,k,pad_idx_q,pad_idx_k):
        len_q,len_k=q.size(1),k.size(1)#.size和.shape有什么区别
#         # (Batch, Time, len_q, len_k)
#         * `q.ne(pad_idx_q)`: 检查`q`中的每个元素是否不等于`pad_idx_q`，返回一个布尔张量，其中True表示非填充元素，False表示填充元素。  
# * `unsqueeze(1)`和`unsqueeze(3)`: 在第二维和第四维上增加一个维度，使张量的形状从`(Batch, len_q)`变为`(Batch, 1, len_q, 1)`。  
# * `repeat(1, 1, 1, len_k)`: 在第四维上重复张量`len_k`次，使其形状变为`(Batch, 1, len_q, len_k)`。
        q = q.ne(pad_idx_q).unsqueeze(1).unsqueeze(3)
        q = q.repeat(1, 1, 1, len_k)

        k = k.ne(pad_idx_k).unsqueeze(1).unsqueeze(2)
        k = k.repeat(1, 1, len_q, 1)

        mask = q & k
        return mask

    def make_causal_mask(self, q, k):
        len_q, len_k = q.size(1), k.size(1)
        mask = (
        # """
        # * `torch.ones(len_q, len_k)`: 创建一个形状为`(len_q, len_k)`的全1矩阵。  
        # * `torch.tril(...)`: 使用`torch.tril`函数，我们保留矩阵的下三角部分（包括对角线），并将其余部分设置为0。在因果掩码中，下三角部分（包括对角线）为True，表示这些位置上的元素在计算自注意力时可以“看到”或“注意”到；而上三角部分为False，表示在计算当前位置的注意力时，不能“看到”未来的位置。  
        # * `.type(torch.BoolTensor)`: 将结果矩阵的数据类型转换为布尔类型（True/False）。  
        # * `.to(self.device)`: 将掩码移动到与类实例关联的设备上（可能是CPU或某个GPU）。
        # """
            torch.tril(torch.ones(len_q, len_k)).type(torch.BoolTensor).to(self.device)
        )
        return mask

    def forward(self, src, trg):
        src_mask = self.make_pad_mask(src, src, self.src_pad_idx, self.src_pad_idx)
        trg_mask = self.make_pad_mask(
            trg, trg, self.trg_pad_idx, self.trg_pad_idx
        ) * self.make_causal_mask(trg, trg)#按元素相乘
        src_trg_mask = self.make_pad_mask(trg, src, self.trg_pad_idx, self.src_pad_idx)

        enc = self.encoder(src, src_mask)
        ouput = self.decoder(trg, enc, trg_mask, src_trg_mask)
        return ouput
    




hasattr是什么，是哪里import进来的函数

In [14]:
def initialize_weights(m):
    if hasattr(m, "weight") and m.weight.dim()>1:
        nn.init.kaiming_uniform(m.weight.data)

        

In [16]:
if __name__ == "__main__":
    enc_voc_size = 5893
    dec_voc_size = 7853
    src_pad_idx = 1
    trg_pad_idx = 1
    trg_sos_idx = 2
    batch_size = 128
    max_len = 1024
    d_model = 512
    n_layers = 3
    n_heads = 2
    ffn_hidden = 1024
    drop_prob = 0.1
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = Transformer(
        src_pad_idx=src_pad_idx,
        trg_pad_idx=trg_pad_idx,
        d_model=d_model,
        enc_voc_size=enc_voc_size,
        dec_voc_size=dec_voc_size,
        max_len=max_len,
        ffn_hidden=ffn_hidden,
        n_heads=n_heads,
        n_layers=n_layers,
        drop_prob=drop_prob,
        device=device,
    ).to(device)

    model.apply(initialize_weights)
    src = torch.load("tensor_src.pt")
    src = torch.cat((src, torch.ones(src.shape[0], 2, dtype=torch.int)), dim=-1)
    trg = torch.load("tensor_trg.pt")

    result = model(src, trg)
    print(result)
    print("above is result")
    print(result.shape)
    print("------done-------")

/tmp/ipykernel_3954/198369884.py:3: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  nn.init.kaiming_uniform(m.weight.data)


tensor([[[ 0.0182,  0.8573,  2.1676,  ..., -1.0181, -0.8391,  0.6301],
         [-0.1038,  0.8671,  2.4428,  ...,  0.0572,  0.7169,  2.1889],
         [-0.7449,  0.9061,  2.1292,  ..., -0.6728, -0.5952,  0.2645],
         ...,
         [-0.0817,  1.5581,  2.5416,  ..., -0.0180, -1.6679, -0.5903],
         [-0.3085,  1.6497,  1.8682,  ...,  1.1634, -0.6081,  1.2884],
         [-0.6122,  1.1131,  1.8740,  ...,  1.6652, -0.4030,  0.0614]],

        [[-0.2470,  1.4090,  1.5266,  ..., -0.3441, -1.0733, -0.0614],
         [ 0.5967,  2.1597,  2.0489,  ..., -0.5584,  0.2949,  0.1743],
         [ 0.1965,  1.4333,  3.0105,  ...,  0.1107,  0.6705, -0.1335],
         ...,
         [-1.9299,  2.6737,  2.8277,  ...,  0.1166,  0.6657, -0.1451],
         [ 0.3026,  1.4098,  1.8824,  ...,  0.6737,  1.4850,  0.1671],
         [-0.4802,  1.9959,  1.8069,  ..., -0.3137,  0.6142, -0.6528]],

        [[-0.1369,  1.4830,  2.1619,  ..., -0.1178, -0.1747, -0.3410],
         [-0.1441,  1.7695,  3.0233,  ..., -0